# WEEK 4 Exercises - Making Queries

![ERD-E-COMMERCE](https://github.com/it-web-pro/django-week4/blob/main/images/WEEK3-ERD(e-commerce).png?raw=true)

## Instruction

1. สร้าง `virtual environment`
2. ติดตั้ง `django` และ `psycopg2` libraries
3. สร้างโปรเจคใหม่ใหม่ชื่อ`myshop`
4. จากนั้นให้ทำการ startapp ใหม่ชื่อ `shop`
5. สร้าง database ชื่อ `shop` ใน Postgres DB
6. ทำการเพิ่ม code ด้านล่างนี้ในไฟล์ `shop/models.py`
7. เพิ่ม **'shop'** ใน `settings.py`
8. ทำการ `makemigrations` และ `migrate`

```python
from django.db import models

# Create your models here.


class Customer(models.Model):
    first_name = models.CharField(max_length=150)
    last_name = models.CharField(max_length=200)
    email = models.CharField(max_length=150)
    address = models.JSONField(null=True)

class ProductCategory(models.Model):
    name = models.CharField(max_length=150)

class Product(models.Model):
    name = models.CharField(max_length=150)
    description = models.TextField(null=True, blank=True)
    remaining_amount = models.PositiveIntegerField(default=0)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    categories = models.ManyToManyField(ProductCategory)

class Cart(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    create_date = models.DateTimeField()
    expired_in = models.PositiveIntegerField(default=60)
    
class CartItem(models.Model):
    cart = models.ForeignKey(Cart, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Order(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    order_date = models.DateField()
    remark = models.TextField(null=True, blank=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    payment_date = models.DateField()
    remark = models.TextField(null=True, blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)

class PaymentItem(models.Model):
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    order_item = models.OneToOneField(OrderItem, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    
class PaymentMethod(models.Model):
    class MethodChoices(models.Choices):
        QR = "QR"
        CREDIT = "CREDIT"
    
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    method = models.CharField(max_length=15, choices=MethodChoices.choices)
    price = models.DecimalField(max_digits=10, decimal_places=2)
```

**จากนั้นให้ทำการ migrate และ run คำสั่งในไฟล์ `shop.sql` ใน PgAdmin4**

In [4]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

### 1. ให้นักศึกษา Query ค้นหาข้อมูลมาแสดงให้ถูกต้องตามโจทย์

1.1 query หาข้อมูล `Order` ทั้งหมดที่เกิดขึ้นในเดือน `พฤษภาคม` และ`ราคา`ของ Order มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
ORDER ID:22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID:23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID:24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID:25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID:26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID:27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID:28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID:29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID:30, DATE: 2024-05-03, PRICE: 1690.00
ORDER ID:31, DATE: 2024-05-04, PRICE: 3280.00
```

In [6]:
from shop.models import *
order = Order.objects.filter(order_date__month=5)[:10]
for i in order:
    print("ORDER ID:%d, DATE: %s, "%(i.id, i.order_date), end="")
    
    orderItem = OrderItem.objects.filter(order=i)
    price = 0
    for j in orderItem:
        price += j.amount * j.product.price
    print("PRICE: %.2f"%price)

<QuerySet [<Order: Order object (22)>, <Order: Order object (23)>, <Order: Order object (24)>, <Order: Order object (25)>, <Order: Order object (26)>, <Order: Order object (27)>, <Order: Order object (28)>, <Order: Order object (29)>, <Order: Order object (30)>, <Order: Order object (31)>]>
ORDER ID:22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID:23, DATE: 2024-05-01, PRICE: 5080.00
ORDER ID:24, DATE: 2024-05-01, PRICE: 3140.00
ORDER ID:25, DATE: 2024-05-02, PRICE: 642499.00
ORDER ID:26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID:27, DATE: 2024-05-02, PRICE: 3570.00
ORDER ID:28, DATE: 2024-05-03, PRICE: 27499.00
ORDER ID:29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID:30, DATE: 2024-05-03, PRICE: 2680.00
ORDER ID:31, DATE: 2024-05-04, PRICE: 9660.00


1.2 query หาข้อมูล `Product` ที่มีคำลงท้ายว่า `features.` ในรายละเอียดสินค้า และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.
```

In [19]:
from shop.models import *
products = Product.objects.exclude(description__endswith="features.")
for i in products:
    print("PRODUCT ID:%d, DESCRIPTION: %s"%(i.id, i.description))

PRODUCT ID:2, DESCRIPTION: A lightweight and high-performance laptop for professionals.
PRODUCT ID:3, DESCRIPTION: An ultra-HD smart television with streaming capabilities.
PRODUCT ID:4, DESCRIPTION: Wireless earphones with noise-canceling technology.
PRODUCT ID:5, DESCRIPTION: A compact tablet for entertainment and productivity.
PRODUCT ID:6, DESCRIPTION: A powerful gaming console with immersive graphics.
PRODUCT ID:8, DESCRIPTION: High-speed wireless router for home or office use.
PRODUCT ID:9, DESCRIPTION: Compact and portable power bank for charging devices on the go.
PRODUCT ID:11, DESCRIPTION: Energy-efficient refrigerator with ample storage space.
PRODUCT ID:12, DESCRIPTION: Front-loading washing machine with multiple wash programs.
PRODUCT ID:13, DESCRIPTION: Compact microwave oven with defrost and cooking functions.
PRODUCT ID:15, DESCRIPTION: Powerful vacuum cleaner for efficient cleaning.
PRODUCT ID:16, DESCRIPTION: Fast-boiling electric kettle with auto shut-off feature.
PR

1.3 query หาข้อมูล `Product` ที่มีราคาสินค้าตั้งแต่ `5000.00` ขึ้นไป และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID: 14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID: 31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID: 54, NAME: Automatic Pet Feeder, PRICE: 7900.00
PRODUCT ID: 61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID: 62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID: 63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID: 64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID: 65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00
```

In [38]:
from shop.models import *
products = Product.objects.filter(price__gte=5000)
for i in products:
    print("PRODUCT ID:%d, NAME: %s, PRICE: %.2f"%(i.id, i.name, i.price))

PRODUCT ID:1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID:2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID:3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID:5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID:6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID:7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID:11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID:14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID:31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID:61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID:62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID:63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID:64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID:65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00
PRODUCT ID:69, NAME: Notebook HP Pavilion Silver, PRICE: 20000.00
PRODUCT ID:72, NAME: Notebook HP Pavilion Silver, PRICE: 20000.00
PRODUCT ID:75, NAME: Notebook HP Pavilion Silver, PRICE: 20000.00


1.4 query หาข้อมูล `Product` ที่มีราคาสินค้าน้อยกว่า `200.00` และมากกว่า `100.00` และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
```

In [22]:
from shop.models import *
products = Product.objects.filter(price__range=(100, 200))
for i in products:
    print("PRODUCT ID:%d, NAME: %s, PRICE: %.2f"%(i.id, i.name, i.price))

PRODUCT ID:21, NAME: Men's T-Shirt, PRICE: 200.00
PRODUCT ID:28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID:29, NAME: Unisex Cap, PRICE: 200.00
PRODUCT ID:49, NAME: Baby Einstein Take Along Tunes Musical Toy, PRICE: 200.00


### 2. เพิ่ม ลบ แก้ไข สินค้า

#### หมวดหมู่สินค้า
- Information technology
- Electronics
- Clothing and Apparel
- Home Appliances
- Furniture
- Toys and Games
- Books and Media
- Pet Supplies
- Jewelry

In [23]:
for i in ProductCategory.objects.all():
    print(i.name)

Information Technology
Electronics
Clothing and Apparel
Home Appliances
Furniture
Toys and Games
Pet Supplies
Jewelry
Books


2.1 ให้เพิ่มสินค้าใหม่จำนวน 3 รายการ (0.5 คะแนน)

```txt
สินค้าที่ 1
ชื่อ: Philosopher's Stone (1997)
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 20
รายละเอียดซ: By J. K. Rowling.
ราคา: 790

สินค้าที่ 2
ชื่อ: Me Before You
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 40
รายละเอียดซ: A romance novel written by Jojo
ราคา: 390

สินค้าที่ 3
ชื่อ: Notebook HP Pavilion Silver
หมวดหมู่สินค้า: Information Technology และ Electronics
จำนวนคงเหลือ: 10
รายละเอียดซ: Display Screen. 16.0
ราคา: 20000
```

In [33]:
from shop.models import *
books_category = ProductCategory.objects.get(pk=7)
it_category = ProductCategory.objects.get(pk=1)
eletro_category = ProductCategory.objects.get(pk=2)
cate8gory = ProductCategory.objects.get(pk=8)

product_1 = Product.objects.create(name="Philosopher's Stone (1997)", remaining_amount=20, description="By J. K. Rowling.", price=790)
product_1.categories.add(books_category)

product_2 = Product.objects.create(name="Me Before You", remaining_amount=40, description="A romance novel written by Jojo", price=390)
product_2.categories.add(books_category)

product_3 = Product.objects.create(name="Notebook HP Pavilion Silver", remaining_amount=10, description="Display Screen. 16.0", price=20000)
product_3.categories.add(it_category)
product_3.categories.add(eletro_category)

2.2 แก้ไขชื่อสินค้า จาก `Philosopher's Stone (1997)` เป็น `Half-Blood Prince (2005)` (0.5 คะแนน)

In [34]:
product_1.name = "Half-Blood Prince (2005)"
product_1.save()

2.3 แก้ไขชื่อหมวดหมู่สินค้า จาก `Books and Media` เป็น `Books` (0.5 คะแนน)

In [35]:
books_category.name = "Books"
books_category.save()

2.4 ลบสินค้าทุกตัวที่อยู่ในหมวดหมู่ `Books` (0.5 คะแนน)

In [37]:
Product.objects.filter(categories=books_category).delete()

(4, {'shop.Product_categories': 2, 'shop.Product': 2})

In [58]:
from django.db.models import Q
pro = Product.objects.filter(Q(description__contains="for")|
                             Q(description__contains="or") | 
                             Q(description__contains="and"), price__range=(500, 50000))
for i in pro:
    print("ID:%d, DESCRIPTION:%s, PRICE:%d"%(i.id, i.description, i.price))

ID:1, DESCRIPTION:A sleek and powerful smartphone with advanced features., PRICE:5900
ID:2, DESCRIPTION:A lightweight and high-performance laptop for professionals., PRICE:25999
ID:5, DESCRIPTION:A compact tablet for entertainment and productivity., PRICE:12900
ID:8, DESCRIPTION:High-speed wireless router for home or office use., PRICE:990
ID:10, DESCRIPTION:A stylish smartwatch with health monitoring and notification features., PRICE:4500
ID:11, DESCRIPTION:Energy-efficient refrigerator with ample storage space., PRICE:9000
ID:13, DESCRIPTION:Compact microwave oven with defrost and cooking functions., PRICE:1290
ID:14, DESCRIPTION:Split air conditioner with remote control and energy-saving features., PRICE:18900
ID:15, DESCRIPTION:Powerful vacuum cleaner for efficient cleaning., PRICE:4000
ID:20, DESCRIPTION:Steam iron with non-stick soleplate for wrinkle-free clothes., PRICE:1200
ID:24, DESCRIPTION:Elegant knee-length dress for special occasions., PRICE:3200
ID:25, DESCRIPTION:Classi